# Extract

In [1]:
import pymssql
import pandas as pd
import json
import multiprocessing as mp
import datetime
import pymysql

/home/adarsh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """Entry point for launching an IPython kernel.


In [4]:
server = '192.168.4.117'
database = 'FreedomCashLenders'
username = 'FreedomCashLendersAll'
mssql_password = 'Freedom123$'


In [5]:
iloans_conn = pymssql.connect(server, username, mssql_password, database, port = 1433)

In [ ]:
start_date = "'2018-01-01'"
end_date = "'2019-12-31'"

In [9]:
query_loan = f'''select LN.LoanId,
                       LC.LoanCount,
                       LN.OriginationDate,
                       GC.BankReportData,
                       GC.TimeAdded as ReportTimeAdded,
                       LN.Campaign,
                       LN.MonthlyGrossIncome,
                       LN.DateOfBirth,
                       LN.IsFirstDefault
                       
                from view_FCL_Loan LN
                LEFT JOIN view_FCL_CustomerLoanCount LC ON LC.CustomerId = LN.CustomerId
                LEFT JOIN view_FCL_GetCreditDataLoan GCD ON LN.LoanId = GCD.LoanId
                LEFT JOIN view_FCL_GetCreditData GC ON GC.BankTransactionId = GCD.BankTransactionId
                
                
                where LN.OriginationDate >= {start_date}
                and LN.OriginationDate <= {end_date} 
                and LN.IsFirstDefault IS NOT NULL
                and LN.MerchantId IN (15, 18)
                and GC.ReportStatus = 'COMPLETE' '''

In [18]:
df_loans = pd.read_sql_query(query_loan,con = iloans_conn)

In [19]:
df_loans = df_loans.drop_duplicates('LoanId')

In [ ]:
query_esign = f'''
SELECT
    LN.LoanId,
    ESIG.AccessCount,
    ESIG.EsigTimeSignedDiff_In_SEC
FROM
    view_FCL_Loan LN
    LEFT JOIN view_FCL_EsignatureCustomerData ESIG ON LN.LoanId = ESIG.LoanId
WHERE
    LN.OriginationDate >= {start_date} 
    and LN.OriginationDate <= {end_date}
    and LN.IsFirstDefault IS NOT NULL
    and LN.MerchantId IN (15, 18)

'''


In [ ]:
df_esign = pd.read_sql_query(query_esign,con=iloans_conn)

## Extract Data for Evaluation

In [ ]:
def stringify_account_ids(loan_id_list):
    """
    Convert account_id list into comma-separated string of account_ids
    :return: string containing comma-separated account_ids
    """
    return '(' + ', '.join([str(i) for i in loan_id_list]) + ')'

In [ ]:
loanid_string=stringify_account_ids(loan_id_list)

In [ ]:
query_loan_eval = '''select LN.LoanId,
                       LC.LoanCount,
                       LN.OriginationDate,
                       GC.BankReportData,
                       GC.TimeAdded as ReportTimeAdded,
                       LN.Campaign,
                       LN.MonthlyGrossIncome,
                       LN.DateOfBirth,
                       LN.IsFirstDefault
                       
                from view_FCL_Loan LN
                LEFT JOIN view_FCL_CustomerLoanCount LC ON LC.CustomerId = LN.CustomerId
                LEFT JOIN view_FCL_GetCreditDataLoan GCD ON LN.LoanId = GCD.LoanId
                LEFT JOIN view_FCL_GetCreditData GC ON GC.BankTransactionId = GCD.BankTransactionId
                
                
                WHERE
                    GC.ReportStatus = 'COMPLETE'
                    AND LN.LoanId IN %s'''%(loanid_string)

In [ ]:
df_loan_eval = pd.read_sql_query(query_loan_eval,con=iloans_conn)

In [ ]:
df_loan_eval=df_loan_eval.drop_duplicates('LoanId')

In [ ]:
query_esign_eval = '''
SELECT
    LN.LoanId,
    ESIG.AccessCount,
    ESIG.EsigTimeSignedDiff_In_SEC
FROM
    view_FCL_Loan LN
    LEFT JOIN view_FCL_EsignatureCustomerData ESIG ON LN.LoanId = ESIG.LoanId
WHERE
    LN.LoanId IN %s'''%(loanid_string)

In [ ]:
df_esign_eval = pd.read_sql_query(query_esign_eval,con=iloans_conn)

In [ ]:
df_esign_eval=df_esign_eval.drop_duplicates('LoanId')

# Preprocess

## utility functions

In [8]:
def parse_dates(json_date):
    '''
    Converts json formatted date to pandas datetime.
    
    Parameters:
    JSON date (JSON).
    
    Returns:
    Pandas datetime object.
    
    '''
    
    #return datetime.fromtimestamp(int(json_date)/1000.0).strftime('%Y-%m-%d')
    return datetime.datetime.utcfromtimestamp(int(json_date)/1000).date()


def fetch_checking_acct_txns(json_string):
    """
    Parse all checking account transactions in the bank report
    
    Parameters:
    json_string(json): json containing bank report
    
    Returns:
    dataframe: containing transactions 
    
    """
    j = json.loads(json_string)
    df_txn = pd.DataFrame()
    
    acct_numbers = []
    for accts in j['accounts']:
        
        if ('transactions' in accts.keys()) and (len(accts['transactions']) > 0) and (accts['accountNumber'] not in acct_numbers) and (accts['accountType'].strip().lower() == 'checking'):
            
            df_txn_temp = pd.DataFrame(accts['transactions'])
            df_txn_temp['account_number'] = accts['accountNumber']
            df_txn = df_txn.append(df_txn_temp, ignore_index=True)
            
            df_txn['posted_date'] = df_txn['postedDate'].map(lambda json_date: parse_dates(json_date))
            df_txn['category'] = df_txn['contexts'].map(lambda x: x[0]['categoryName'] if len(x) > 0 else np.nan)
            acct_numbers.append(accts['accountNumber'])
    
    if 'pending' in df_txn.columns:
        df_txn = df_txn[df_txn['pending'] == False]
    return df_txn

## primary account

In [9]:
def get_primary_account(bankreport):
    """
    Flag primary checking account (account having max transaction count)
    
    Parameters:
    bankreport (json)
    loanid (str)
    
    Returns:
    Dataframe containing checking accounts and primary account flag = 1
    """
    df_txn = fetch_checking_acct_txns(bankreport)
    if df_txn.empty is False:
        df_txns_count = df_txn['account_number'].value_counts()
        return df_txns_count.idxmax()

In [ ]:
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    res_primary_accts = pool.map(get_primary_account, df_loans['BankReportData'])

In [21]:
df_loans['primary_account'] = res_primary_accts

In [ ]:
df_loans = df_loans.loc[df_loans['primary_account'].notnull(),:]

## filter loans having transaction days >= 60 in primary account

In [22]:
def get_transaction_days_count(primary_account,bank_report):
    df_checking_txns = fetch_checking_acct_txns(bank_report)
    if df_checking_txns.empty is False:
        df_primary_account_txns = df_checking_txns[df_checking_txns['account_number']==primary_account]
        df_primary_account_txns= df_primary_account_txns.sort_values(by='posted_date')
        first_txn_date = df_primary_account_txns['posted_date'].iloc[0]
        last_txn_date = df_primary_account_txns['posted_date'].iloc[-1]
        txn_days_count = (last_txn_date - first_txn_date).days
        return txn_days_count >= 60

In [ ]:
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    txn_days_count = pool.starmap(get_transaction_days_count, zip(df_loans['primary_account'],df_loans['BankReportData']))

In [24]:
df_loans['txn_days_count'] = txn_days_count

In [ ]:
df_loans = df_loans.loc[df_loans['txn_days_count'] == True, :]

## Calculate Age

In [14]:
def calculate_age(current_date, dob):
    age = len(pd.date_range(start=dob,end=current_date,freq='Y'))
    return age

In [25]:
df_loans['Age'] = df_loans.apply(lambda x: calculate_age(x['OriginationDate'],x['DateOfBirth']), axis = 1)

## New or Reloan

In [26]:
df_loans['Reloan'] = df_loans['LoanCount'].apply(lambda x:True if x>1 else False)

### Lead Provider

In [ ]:
df_loans = df_loans.loc[df_loans['Campaign'].notnull(),:]

In [ ]:
lead_provider_list=['MarketBullet','StopNGo','Nimbus','EPCVIP','PingBid','LeapThry',
'Acquir','RoundSky','Zero','LeadPie',
'ITMedia','LeadsMarket']

In [ ]:
df_loans['LeadProvider'] = df_loans['Campaign'].str.extract("(" + "|".join(lead_provider_list) +")",flags = re.IGNORECASE)

In [ ]:
df_loans['LeadProvider']=df_loans['LeadProvider'].fillna('Freedom')

### lender vars

In [ ]:
def create_lender_vars(loanid,report_string,time_added,pr_acct):

        """
        Function to generate lender variables 
        from primary account transactions

        Paramaters:
            txns(Boolean): True - Return lender txns along with lender variables
                           False - Return only lender variables

        Returns:
            lender_vars(dictionary): Dictionary containing all lender variables
        
        """

        lender_vars = dict()   
        lender_vars['LoanId'] = loanid
        lender_vars['LenderAmountDeb'] = 0.0
        lender_vars['LenderCountCred'] = 0.0
        lender_vars['LenderAmountCred30'] = 0.0
        lender_vars['LenderCountDeb'] = 0.0
        lender_vars['LenderAmountDeb30'] = 0.0
        lender_vars['LenderCountCred30'] = 0.0
        lender_vars['LenderCountDeb30'] = 0.0
        lender_vars['LenderAmountCred'] = 0.0
        lender_vars['UniqLenderCount'] = 0.0

        #load lending company list
        lend_cos=joblib.load('./lend_cos.pkl')

        #get primary checking account transactions
        df_checking_txns = fetch_checking_acct_txns(report_string) 
        df_pr_acct_txns = df_checking_txns[df_checking_txns['account_number']==pr_acct]
        
        
        #prepare lender transactions dataframe
        df_lender_txns=df_pr_acct_txns.loc[df_pr_acct_txns['memo'].str.contains('|'.join(lend_cos),case=False,na=False)]
        
        #check for empty transactions
        if df_lender_txns.empty is False:
            df_lender_txns['lenderName'] = df_lender_txns['memo'].str.extract("(" + "|".join(lend_cos) +")",flags = re.IGNORECASE)
            df_lender_txns['days_diff'] = (time_added.date()-df_lender_txns['posted_date']).dt.days
            df_lender_txns['amount'] = df_lender_txns['amount'].round(2)


            #conditions to determine lender variables
            cond1 = (df_lender_txns['amount']>0)
            cond2 = cond1 & (df_lender_txns['days_diff']<=30)
            cond3 = (df_lender_txns['amount']<0)
            cond4 = cond3 & (df_lender_txns['days_diff']<=30)

            #prepare lender variables
            lender_vars['LenderAmountDeb'] = float(df_lender_txns.loc[cond3,'amount'].sum())
            lender_vars['LenderCountCred'] = float(df_lender_txns[cond1].shape[0])
            lender_vars['LenderAmountCred30'] = float(df_lender_txns.loc[cond2,'amount'].sum())
            lender_vars['LenderCountDeb'] = float(df_lender_txns[cond3].shape[0])
            lender_vars['LenderAmountDeb30'] = float(df_lender_txns.loc[cond4,'amount'].sum())
            lender_vars['LenderCountCred30'] = float(df_lender_txns.loc[cond2].shape[0])
            lender_vars['LenderCountDeb30'] = float(df_lender_txns.loc[cond4].shape[0])
            lender_vars['LenderAmountCred'] = float(df_lender_txns.loc[cond1,'amount'].sum())
            lender_vars['UniqLenderCount'] = float(df_lender_txns['lenderName'].nunique())

        return pd.DataFrame(lender_vars,index=[0])
    

#### in case the lender vars are to be generated for funded loans between 2018-01-01 to 2019-12-31 do not run the below cell, instead download from s3 ( look for "download lender vars from s3" markdown)

In [ ]:
df_lender_vars = pd.DataFrame()
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    df_lender_vars_temp = pool.starmap(create_lender_vars, zip(df_loans['LoanId'],df_loans['BankReportData'],df_loans['ReportTimeAdded'],df_loans['primary_account']))
df_lender_vars=pd.concat(df_lender_vars_temp,ignore_index=True)

In [ ]:
df_lender_vars.reset_index(drop=True,inplace=True)

#### download lender vars from s3

In [ ]:
import boto3

In [ ]:
#provide access keys if needed
s3 = boto3.client('s3')

In [ ]:
s3.download_file('predicon-bucket', 'lender_vars.csv', 'FILE_NAME')

In [ ]:
df_lender_vars = pd.read_csv('lender_vars.csv')

In [ ]:
df_loans = pd.merge(df_loans,df_lender_vars,how='left',on='LoanId')

### esign variables

In [ ]:
df_loans= pd.merge(df_loans,df_esign,on='LoanId',how='left')

# Train

## useful links
https://docs.databricks.com/_static/notebooks/mlflow/mlflow-quick-start-deployment-aws.html

https://towardsdatascience.com/deploying-models-to-production-with-mlflow-and-amazon-sagemaker-d21f67909198

https://www.h2o.ai/blog/a-deep-dive-into-h2os-automl/

## specify features

In [ ]:
features_drop = ['LoanCount',
'OriginationDate',             
'BankReportData',                   
'ReportTimeAdded',                  
'Campaign',
'primary_account',
'txn_days_count', 'DateOfBirth',]

In [ ]:
df_train = df_loans.drop(columns=features_drop,axis=1)

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(max_mem_size='16G')

In [ ]:
df_h20_train =  h2o.H2OFrame(df_train)

In [ ]:
y = "IsFirstDefault" 
x = df_h20_train.columns
x.remove(y)
x.remove('LoanId')

In [ ]:
aml = H2OAutoML(max_runtime_secs=120, seed=1)
aml.train(x=x, y=y, training_frame=df_h20_train)

In [ ]:
lb = aml.leaderboard
lb.head()

# Predict

### primary account

In [ ]:
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    res_primary_accts = pool.map(get_primary_account, df_loan_eval['BankReportData'])

In [ ]:
df_loan_eval['primary_account'] = res_primary_accts

### filter loans having transaction days >= 60 in primary account

In [ ]:
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    txn_days_count = pool.starmap(get_transaction_days_count, zip(df_loan_eval['primary_account'],df_loan_eval['BankReportData']))

In [ ]:
df_loan_eval['txn_days_count'] = txn_days_count

In [ ]:
df_loan_eval = df_loan_eval.loc[df_loan_eval['txn_days_count']==True,:]

### calculate age

In [ ]:
df_loan_eval['Age'] = df_loan_eval.apply(lambda x: calculate_age(x['OriginationDate'],x['DateOfBirth']), axis = 1)

### lead provider

In [ ]:
df_loan_eval = df_loan_eval.loc[df_loan_eval['Campaign'].notnull(),:]

In [ ]:
df_loan_eval['LeadProvider'] = df_loan_eval['Campaign'].str.extract("(" + "|".join(lead_provider_list) +")",flags = re.IGNORECASE)

In [ ]:
df_loan_eval['LeadProvider']=df_loan_eval['LeadProvider'].fillna('Freedom')

### new or reloan

In [ ]:
df_loan_eval['Reloan'] = df_loan_eval['LoanCount'].apply(lambda x:True if x>1 else False)

### lender vars

In [ ]:
df_lender_vars_eval = pd.DataFrame()
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    df_lender_vars_temp = pool.starmap(create_lender_vars, zip(df_loan_eval['LoanId'],df_loan_eval['BankReportData'],df_loan_eval['ReportTimeAdded'],df_loan_eval['primary_account']))
df_lender_vars_eval=pd.concat(df_lender_vars_temp,ignore_index=True)

In [ ]:
df_loan_eval = pd.merge(df_loan_eval,df_lender_vars_eval,on='LoanId',how='left')

### esign variables

In [ ]:
df_loan_eval= pd.merge(df_loan_eval,df_esign_eval,on='LoanId',how='left')

### run prediction

In [ ]:
df_loan_predict = df_loan_eval.drop(columns=features_drop,axis=1)

In [ ]:
h2o_eval = h2o.H2OFrame(df_loan_predict)

In [ ]:
pred = aml.leader.predict(h2o_eval)
pred.head()

In [ ]:
#convert to pandas dataframe
df_predictions = h2o.as_list(pred)

In [ ]:
df_loan_eval.reset_index(drop=True,inplace=True)

In [ ]:
df_predictions['target'] = df_loan_eval['IsFirstDefault']  

In [ ]:
df_predictions = df_predictions.rename(columns={'True':'prob'})

In [ ]:
df_predictions = df_predictions[['target','prob']]

In [ ]:
df_predictions['target'].value_counts(normalize = True)

# Evaluate

## get BV uncertain and BV Approved loans for model evaluation

In [28]:
username_bank_app = 'bankreview'
password_bank_app = 'Freedom!23'
host_bank_app = '192.168.4.115'
port_bank_app = 3306
db_bank_app = 'bankreviewdb'

In [29]:
bank_app_conn = pymysql.connect(host=host_bank_app,
                                port=port_bank_app,
                                db=db_bank_app,
                                user=username_bank_app,
                                password=password_bank_app)

In [30]:
query_evaluation_loans = '''select loan_id, 
                                final_decision,
                                reasons_for_decision,
                                entered_date
                                
                            from loan 
                            where campaign like '%Production%'
                            and STR_TO_DATE(entered_date ,'%m/%d/%Y') >= STR_TO_DATE('01/01/2020','%m/%d/%Y')
                            and STR_TO_DATE(entered_date ,'%m/%d/%Y') < STR_TO_DATE('04/01/2020','%m/%d/%Y')
                            and final_decision in ('Bank Validation Uncertain','Bank Validation Approved') '''

In [ ]:
df_eval_loans = pd.read_sql_query(query_evaluation_loans, con = bank_app_conn)

## get funded and mature loans for the same period

In [32]:
query_funded_mature_loans = ''' select LoanId, 
                                IsFirstDefault
                        from view_FCL_Loan
                        where OriginationDate >= '2020-01-01' 
                        and OriginationDate <= '2020-03-31'
                        and IsFirstDefault IS NOT NULL
                        and MerchantId IN (15, 18)
                        
                     '''

In [33]:
df_funded_mature_loans = pd.read_sql_query(query_funded_mature_loans,con = iloans_conn)

In [34]:
df_funded_mature_loans['LoanId'] = df_funded_mature_loans['LoanId'].astype(int).astype(str)

In [35]:
df_eval = pd.merge(df_funded_mature_loans,df_eval_loans,how = 'inner',left_on = 'LoanId',right_on = 'loan_id')

In [36]:
df_eval.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1199 entries, 0 to 1198
Data columns (total 6 columns):
LoanId                  1199 non-null object
IsFirstDefault          1199 non-null bool
loan_id                 1199 non-null object
final_decision          1199 non-null object
reasons_for_decision    1199 non-null object
entered_date            1199 non-null object
dtypes: bool(1), object(5)
memory usage: 57.4+ KB


In [ ]:
loan_id_list = list(df_eval['LoanId'])

### compute KS

In [ ]:
import numpy as np

In [ ]:
def get_KS(df_pred):
    """
    Returns KS given scores
    Parameters:
    df_pred (pandas df): DataFrame containing target variable and model score
    
    Returns:
    float: KS value
    """
    df_scores = df_predictions.sort_values(by='prob')
    total_good = (df_scores['target'] == False).sum()
    total_bad = (df_scores['target'] == True).sum()
    df_scores['cum_good_perc'] = (df_scores['target'] == False).cumsum()/total_good
    df_scores['cum_bad_perc'] = (df_scores['target'] == True).cumsum()/total_bad
    df_scores['cum_diff'] = np.abs((df_scores['cum_good_perc'] - df_scores['cum_bad_perc']))
    return df_scores['cum_diff'].max()

In [ ]:
get_KS()

### quantiling

In [ ]:
def quantile_table(df_pred,n = 10):
    """
    Returns a quantile table given model scores (default is decile)
    
    Parameters:
    df_pred (pandas df): DataFrame containing target variable and model score
    
    Returns:
    pandas DataFrame: Pandas dataframe containing quantiles
    
    """
    df_scores = df_predictions.sort_values(by='prob')
    df_scores['decile'],score_bin = pd.qcut(df_scores['prob'],10,labels=[1,2,3,4,5,6,7,8,9,10],retbins = True)
    df_scores['target'] = df_scores['target'].astype(int)
    df_scores_deciles = df_scores.groupby('decile',as_index=False).agg({'prob':['count','min','max','mean'],'target':'sum'})
    df_scores_deciles.columns = ['decile','count','min_score','max_score','mean_score','bad_count']
    df_scores_deciles['perc_bad'] = (df_scores_deciles['bad_count']/df_scores_deciles['count']) * 100
    return df_scores_deciles,score_bin

In [ ]:
quantile_table, score_bins = quantile_table(df_predictions)

In [ ]:
quantile_table

### get bins for quantile assignment

In [ ]:
score_bins

In [ ]:
score_bins = np.concatenate(([-np.inf], score_bins, [np.inf]))

### upload to sagemaker

In [ ]:
import mlflow

In [ ]:
import mlflow.h2o as mh2o

In [ ]:
import mlflow.sagemaker as mfs

In [ ]:
mh2o.save_model(aml.leader,path="path/to/trained/model")

In [ ]:
region = "us-east-1"
arn = "arn:aws:iam::757719720041:role/Sagemaker"
appname = "h20-mlflow-deploy"
modeluri = "path/to/saved/model" 
image_url = "757719720041.dkr.ecr.us-east-1.amazonaws.com/freedom-pyfunc:latest"

In [ ]:
mfs.deploy(app_name=appname, model_uri=modeluri, instance_type='ml.t2.medium',region_name=region, mode="create",execution_role_arn=arn,image_url=image_url)